# Task - 1 KNN Implementation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import required Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

import collections
from collections import Counter



In [ ]:

# Loading the dataset
full_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_train_val.csv')
# df = full_df[["text"]]
df = full_df
# df["text"] = df["text"].astype(str)
full_df.head()


,label,text
0,0,भीड़ में बहुत हब्सी मिलेंगे
1,0,साले बेवकूफ अपनी मां मक्खियां तो हटा दें
2,0,बुर देदो तो मुह में लंड ले लो तो
3,0,कुत्ता वहा है चिल्ला तू क्यों रहा है
4,1,चाय नहीं पीता हूं मैं इसी को छोड़ दिया ok


In [ ]:
def count_length():
    df['word_count'] = df['text'].apply(lambda x: len(str(x).split(" ")))

In [ ]:
count_length()
df.tail(10)

,label,text,word_count
20173,0,वहीं पर ठोक वाले तू तो अच्छा था ना,9
20174,1,💥💥💥 गुड नाइट जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...,4
20175,0,तेरी माँ की कसम,4
20176,1,"अपना जीवन जीने के दो तरीके है, एक मायने के कुछ...",21
20177,1,तीसरा का नाम् कल्लू है,5
20178,0,"और छोटे भी क्यु पहनते हो, कहा जरूरत है",9
20179,1,कैसी हो सोना क्या आप हमसे दोस्ती करोगी,8
20180,1,इस से ये पता चलता है अब चुनाव आ रहा है🙄🙄🙄,11
20181,1,खेत जोते हुए किसान भैंस के साथो,7
20182,0,काश आगर यह इतना दिमाग पढाई में लगायें तो शायद ...,18


## Data Preprocessing by removing remove unnecessary symbols, stop words,etc.

In [ ]:
!pip install indic_transliteration indicnlp indic-nlp-library indic
# pip install indicnlp
# pip install indic-nlp-library
# pip install indic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.6 MB/s eta 0:00:00
  Created wheel for indic: filename=indic-0.1.2-py3-none-any.whl size=10089 sha256=5547798c2044e218d61fb02c29aaaf46f6764342c3e3747eb8d9c3c2dccca4ea
  Stored in directory: /root/.cache/pip/wheels/99/46/ae/5111371e0c7d7160c689f9f6cee321acd45a4e32d4c419dd4a
Successfully built indic


In [ ]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [ ]:
# stopword list

with open('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/final_stopwords.txt', 'r', encoding="utf8") as file:
    content = file.readlines()

stopwords_hi = [line.strip() for line in content]

# stopwords_hi = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','आगे','माननीय','शहर','बताएं','कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें','आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर', 'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते', 'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन', 'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो', 'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस', 'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे', 'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता', 'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा', 'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि', 'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि', 'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि', 'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 'वरग', 'हुअ', 'जेसा', 'नहिं']
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
# punctuations = ['nn','n', '।','/', '`', '+', '\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}','|','"']

# punctuations = ['(', ')', '?', ':', ';', ',', '.', '!', '/', '"', "'"]

punctuations = [".", ",", "!", "?", ":", ";", "-", "—", "(", ")", "[", "]", "{", "}", "/", "\\", "|", "@", "#", "$", "%", "^", "&", "*", "+", "=", "<", ">", "_", "~", "`", "'", "\"", "«", "»", "“", "”", "‘", "’", "‹", "›", "„", "‟", "‛", "‟", "‘", "’", "‚", "‛", "‟", "‹", "›", "〈", "〉", "《", "》", "「", "」", "『", "』", "【", "】"]



to_be_removed = stopwords_hi + punctuations + stopwords_en

In [ ]:
# removing all stopword

for i in range(len(df)):
    df['text'][i]=[ele for ele in df['text'][i] if ele not in (to_be_removed)]
count_length()
df.tail(10)

,label,text,word_count
20173,0,"[ठोक, तू, अच्छा, ना]",4
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...",4
20175,0,"[तेरी, माँ, कसम]",3
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह...",14
20177,1,"[तीसरा, नाम्, कल्लू]",3
20178,0,"[छोटे, क्यु, पहनते, जरूरत]",4
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]",5
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]",5
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]",6
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा...",13


In [ ]:
# Converting list to string 

# Create a new column called 'processed_text'
df['processed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(df)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(df['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    df.loc[i, 'processed_text'] = text_string

# Call the count_length() function (assuming it's defined somewhere)
count_length()

df.tail(10)


,label,text,word_count,processed_text
20173,0,"[ठोक, तू, अच्छा, ना]",4,ठोक तू अच्छा ना
20174,1,"[💥💥💥, गुड, नाइट, जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...",4,💥💥💥 गुड नाइट जी💥💥💥💥🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂️🙋‍♂...
20175,0,"[तेरी, माँ, कसम]",3,तेरी माँ कसम
20176,1,"[जीवन, जीने, के, तरीके, मायने, के, चमत्कार, नह...",14,जीवन जीने के तरीके मायने के चमत्कार नही दुसरे ...
20177,1,"[तीसरा, नाम्, कल्लू]",3,तीसरा नाम् कल्लू
20178,0,"[छोटे, क्यु, पहनते, जरूरत]",4,छोटे क्यु पहनते जरूरत
20179,1,"[कैसी, सोना, हमसे, दोस्ती, करोगी]",5,कैसी सोना हमसे दोस्ती करोगी
20180,1,"[पता, चलता, चुनाव, आ, है🙄🙄🙄]",5,पता चलता चुनाव आ है🙄🙄🙄
20181,1,"[खेत, जोते, किसान, भैंस, के, साथो]",6,खेत जोते किसान भैंस के साथो
20182,0,"[काश, आगर, इतना, दिमाग, पढाई, लगायें, शायद, जा...",13,काश आगर इतना दिमाग पढाई लगायें शायद जाकर डॉक्ट...


## Hindi to English Translit

In [ ]:
from indic_transliteration import sanscript

def hindi_to_english(text):
    # Set the source and target scripts to be used for transliteration
    source_script = sanscript.DEVANAGARI
    target_script = sanscript.ITRANS
    
    # Transliterate the text from Hindi to English
    result = sanscript.transliterate(text, source_script, target_script)
    
    return result


# Apply the transliteration function to the Hindi text column and create a new column with the English text
df['english_text'] = df['processed_text'].apply(hindi_to_english)

# Print the first 10 rows of the DataFrame to verify that the new column has been added
df.head()



,label,text,word_count,processed_text,english_text
0,0,"[भीड़, हब्सी, मिलेंगे]",3,भीड़ हब्सी मिलेंगे,bhI.Da habsI mileMge
1,0,"[साले, बेवकूफ, मां, मक्खियां, हटा, दें]",6,साले बेवकूफ मां मक्खियां हटा दें,sAle bevakUpha mAM makkhiyAM haTA deM
2,0,"[बुर, देदो, मुह, लंड, ले, लो]",6,बुर देदो मुह लंड ले लो,bura dedo muha laMDa le lo
3,0,"[कुत्ता, वहा, चिल्ला, तू]",4,कुत्ता वहा चिल्ला तू,kuttA vahA chillA tU
4,1,"[चाय, नहीं, पीता, हूं, छोड़, ok]",6,चाय नहीं पीता हूं छोड़ ok,chAya nahIM pItA hUM Cho.Da ok


In [ ]:
# Preprocess

df["text_lower"] = df["english_text"].str.lower()
df.head()

,label,text,word_count,processed_text,english_text,text_lower
0,0,"[भीड़, हब्सी, मिलेंगे]",3,भीड़ हब्सी मिलेंगे,bhI.Da habsI mileMge,bhi.da habsi milemge
1,0,"[साले, बेवकूफ, मां, मक्खियां, हटा, दें]",6,साले बेवकूफ मां मक्खियां हटा दें,sAle bevakUpha mAM makkhiyAM haTA deM,sale bevakupha mam makkhiyam hata dem
2,0,"[बुर, देदो, मुह, लंड, ले, लो]",6,बुर देदो मुह लंड ले लो,bura dedo muha laMDa le lo,bura dedo muha lamda le lo
3,0,"[कुत्ता, वहा, चिल्ला, तू]",4,कुत्ता वहा चिल्ला तू,kuttA vahA chillA tU,kutta vaha chilla tu
4,1,"[चाय, नहीं, पीता, हूं, छोड़, ok]",6,चाय नहीं पीता हूं छोड़ ok,chAya nahIM pItA hUM Cho.Da ok,chaya nahim pita hum cho.da ok


In [ ]:
# drop the new column created in last cell
# df.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text_lower"].apply(lambda text: remove_punctuation(text))
df.head()

,label,text,word_count,processed_text,english_text,text_lower,text_wo_punct
0,0,"[भीड़, हब्सी, मिलेंगे]",3,भीड़ हब्सी मिलेंगे,bhI.Da habsI mileMge,bhi.da habsi milemge,bhida habsi milemge
1,0,"[साले, बेवकूफ, मां, मक्खियां, हटा, दें]",6,साले बेवकूफ मां मक्खियां हटा दें,sAle bevakUpha mAM makkhiyAM haTA deM,sale bevakupha mam makkhiyam hata dem,sale bevakupha mam makkhiyam hata dem
2,0,"[बुर, देदो, मुह, लंड, ले, लो]",6,बुर देदो मुह लंड ले लो,bura dedo muha laMDa le lo,bura dedo muha lamda le lo,bura dedo muha lamda le lo
3,0,"[कुत्ता, वहा, चिल्ला, तू]",4,कुत्ता वहा चिल्ला तू,kuttA vahA chillA tU,kutta vaha chilla tu,kutta vaha chilla tu
4,1,"[चाय, नहीं, पीता, हूं, छोड़, ok]",6,चाय नहीं पीता हूं छोड़ ok,chAya nahIM pItA hUM Cho.Da ok,chaya nahim pita hum cho.da ok,chaya nahim pita hum choda ok


# Preprocessing Test Data set

In [ ]:

# Loading the dataset
tfull_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_test.csv')
# df = full_df[["text"]]
tdf = tfull_df
# df["text"] = df["text"].astype(str)
tfull_df.head()

,label,text
0,0,मैं ये नही सोच रहा की इसे निकले कैसे मैं ये सो...
1,1,और दिवाली में भी पूरा देश पड़ाका नहीं फोडात
2,1,कुत्ता बिल्ली पाल लेना मगर गलत फहमी कभी नहीं।
3,0,तेरी गांड में प्याज काट देगा गुज्जर भोसड़ी के
4,1,बंगाली साड़ी ऐसे नहीं पहना जाता है दीदी


In [ ]:
def count_length():
    tdf['tword_count'] = tdf['text'].apply(lambda x: len(str(x).split(" ")))

count_length()
tdf.tail(10)

,label,text,tword_count
6718,0,चूतियो की कमी नहीं है एक से बढ़कर एक चुतिय है,11
6719,0,प्यार❤ व्यार 💔साला सब धोखा 😡है इसके बिना ही तो...,15
6720,0,नंगी होकर वीडियो बनाओ और अच्छी लगोगी,7
6721,0,यह सही तरीके हैं कपड़े दिखाने के कि मेरे पास क...,13
6722,1,बीजेपी की देन है मंदिर इन सब ने तो मंदिर रुकवा...,19
6723,0,आरे पागल लवडे कुछ तो समज आये वो कर ना,10
6724,0,शेर नज़र अंदाज़ कर रहे हैं। इसे कमजोरी मत समझ ले...,86
6725,0,लाल चड्डी के अंदर भी लाल ही होगा,8
6726,1,कैसे है आप,3
6727,0,पोरीच्या फोटो काय महाराजांचे नाटक का रे मादरचोत,8


In [ ]:


from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
tdf['text'] = tdf['text'].apply(lambda x: tokenization(x))

# removing all stopword

for i in range(len(tdf)):
    tdf['text'][i]=[ele for ele in tdf['text'][i] if ele not in (to_be_removed)]
# count_length()
# tdf.tail(10)

# Converting list to string 

# Create a new column called 'processed_text'
tdf['tprocessed_text'] = ''

# Iterate over each row of the DataFrame
for i in range(len(tdf)):
    # Join the list of strings in the 'text' column with a space separator
    text_string = ' '.join(tdf['text'][i])
    # Remove the stopwords from the text string
    text_string = ' '.join([word for word in text_string.split() if word.lower() not in to_be_removed])
    # Update the 'processed_text' column with the processed text string
    tdf.loc[i, 'tprocessed_text'] = text_string

# Call the count_length() function (assuming it's defined somewhere)
# count_length()

# tdf.tail(10)

from indic_transliteration import sanscript

def hindi_to_english(text):
    # Set the source and target scripts to be used for transliteration
    source_script = sanscript.DEVANAGARI
    target_script = sanscript.ITRANS
    
    # Transliterate the text from Hindi to English
    result = sanscript.transliterate(text, source_script, target_script)
    
    return result


# Apply the transliteration function to the Hindi text column and create a new column with the English text
tdf['tenglish_text'] = tdf['tprocessed_text'].apply(hindi_to_english)

# Print the first 10 rows of the DataFrame to verify that the new column has been added
# tdf.head()

# Preprocess

tdf["ttext_lower"] = tdf["tenglish_text"].str.lower()
# tdf.head()

# drop the new column created in last cell
# df.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

tdf["ttext_wo_punct"] = tdf["ttext_lower"].apply(lambda text: remove_punctuation(text))
tdf.head()


,label,text,tword_count,tprocessed_text,tenglish_text,ttext_lower,ttext_wo_punct
0,0,"[नही, सोच, निकले, सोच, फंसा, कैसे😂😂😂]",17,नही सोच निकले सोच फंसा कैसे😂😂😂,nahI socha nikale socha phaMsA kaise😂😂😂,nahi socha nikale socha phamsa kaise😂😂😂,nahi socha nikale socha phamsa kaise😂😂😂
1,1,"[दिवाली, देश, पड़ाका, नहीं, फोडात]",9,दिवाली देश पड़ाका नहीं फोडात,divAlI desha pa.DAkA nahIM phoDAta,divali desha pa.daka nahim phodata,divali desha padaka nahim phodata
2,1,"[कुत्ता, बिल्ली, पाल, लेना, गलत, फहमी, नहीं, ।]",9,कुत्ता बिल्ली पाल लेना गलत फहमी नहीं ।,kuttA billI pAla lenA galata phahamI nahIM |,kutta billi pala lena galata phahami nahim |,kutta billi pala lena galata phahami nahim
3,0,"[तेरी, गांड, प्याज, काट, देगा, गुज्जर, भोसड़ी,...",9,तेरी गांड प्याज काट देगा गुज्जर भोसड़ी के,terI gAMDa pyAja kATa degA gujjara bhosa.DI ke,teri gamda pyaja kata dega gujjara bhosa.di ke,teri gamda pyaja kata dega gujjara bhosadi ke
4,1,"[बंगाली, साड़ी, नहीं, पहना, दीदी]",8,बंगाली साड़ी नहीं पहना दीदी,baMgAlI sA.DI nahIM pahanA dIdI,bamgali sa.di nahim pahana didi,bamgali sadi nahim pahana didi


## Data spliting fro training and testing

In [ ]:
# Spliting 80% of dataset for traing and 20% for validation

from sklearn.model_selection import train_test_split

# # assuming your data is in X (features) and y (labels) arrays
X=df['text_wo_punct']
y=df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
mc=X_train
bc=X_test

## Extract features from the dataset using TF-IDF (term frequency-inverse document frequency)

In [ ]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Classification using the KNN classification model.

In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=100)
neigh.fit(X_train, y_train)

y_pred=neigh.predict(X_test)
print(y_pred)

[1 1 1 ... 0 1 1]


## Printing accuracy report of KNN

In [ ]:
# Printing accuracy report

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.65      0.74      2140
           1       0.69      0.87      0.77      1897

    accuracy                           0.75      4037
   macro avg       0.77      0.76      0.75      4037
weighted avg       0.77      0.75      0.75      4037



# Saving model for testing

In [ ]:
import joblib


# Save the model as a file
joblib.dump(neigh, 'knn_model.pkl')

# # Load the saved model from a file
# loaded_model = joblib.load('knn_model.pkl')

['knn_model.pkl']

# Loading Model and testing our test dataset and print accuracy report

In [ ]:
tdf.shape

(6728, 7)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load the saved model from a file
loaded_model = joblib.load('knn_model.pkl')

# Load the test data from a CSV file
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/22CS60R37_A8 DL/hindi_test.csv')

# Extract the features (text) and labels from the test data
X_test2 = tdf['ttext_wo_punct']
y_test = tdf['label']


# TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# X_train = vectorizer.fit_transform(X_train)
X_test2 = vectorizer.transform(X_test2)


# # Convert the text features to TF-IDF vectors using the same vectorizer as before
# vectorizer = TfidfVectorizer()
# X_test = vectorizer.transform(X_test)

# Use the loaded model to make predictions on the test data
y_pred = loaded_model.predict(X_test2)

# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.64      0.73      3496
           1       0.69      0.87      0.77      3232

    accuracy                           0.75      6728
   macro avg       0.77      0.76      0.75      6728
weighted avg       0.77      0.75      0.75      6728



# Finding Intersection of test and train data

In [ ]:
def print_intersection_points(train_df, val_df, new_testdf):
  train_com_count = 0
  val_com_count = 0
  for sentence in train_df:
    for new_sentence in new_testdf:
      if sentence==new_sentence:
        train_com_count += 1

  for sentence in val_df:
    for new_sentence in new_testdf:
      if sentence==new_sentence:
        val_com_count += 1
  
  # print("Intersection points for dataset used for testing and training ", modelname)
  print("Common points with training dataset: ",train_com_count)
  print("Common points with validation dataset: ",val_com_count)

print_intersection_points(mc, bc, tdf['ttext_wo_punct'])


Common points with training dataset:  221
Common points with validation dataset:  46
